In [1]:
import pandas as pd
import numpy as np

In [2]:
fin_report = pd.read_excel("Data.xlsx")
fin_report.columns = ["Company", "Date", "Season", "Code", "Industry", "EPS", "PE", "Divdend_Rate"]

fin_report.Date = pd.to_datetime(fin_report.Date)
fin_report["Year"] = fin_report["Date"].apply(lambda x : x.year)
fin_report["Season"] = fin_report["Date"].apply(lambda x : x.quarter)

fin_report.head(17)

,Company,Date,Season,Code,Industry,EPS,PE,Divdend_Rate,Year
0,1101 台泥,2013-03-29,1,1101,水泥工業,0.38,12.60,5.07,2013
1,1101 台泥,2013-06-28,2,1101,水泥工業,0.70,10.52,5.15,2013
2,1101 台泥,2013-09-30,3,1101,水泥工業,0.97,9.71,4.44,2013
3,1101 台泥,2013-12-31,4,1101,水泥工業,0.66,9.38,4.97,2013
4,1101 台泥,2014-03-31,1,1101,水泥工業,0.50,8.79,4.89,2014
5,1101 台泥,2014-06-30,2,1101,水泥工業,0.87,7.80,5.09,2014
6,1101 台泥,2014-09-30,3,1101,水泥工業,0.87,7.99,5.08,2014
7,1101 台泥,2014-12-31,4,1101,水泥工業,0.70,7.62,5.74,2014
8,1101 台泥,2015-03-31,1,1101,水泥工業,0.22,8.53,5.65,2015
9,1101 台泥,2015-06-30,2,1101,水泥工業,0.61,8.96,6.39,2015


In [3]:
#定義函數指定真正的公布日期
def real_announce_day(x):
    if x["Season"] == 1:
        return(pd.to_datetime("{}{}".format(x["Year"], "-05-15")))
    elif x["Season"] == 2:
         return(pd.to_datetime("{}{}".format(x["Year"], "-08-14")))
    elif x["Season"] == 3:
         return(pd.to_datetime("{}{}".format(x["Year"], "-11-14")))
    else:
        return(pd.to_datetime("{}{}".format(x["Year"] + 1, "-03-31")))
    
#執行函數    
fin_report["announce_day"] = fin_report.apply(real_announce_day, axis = 1)

In [4]:
#選擇並排列財報資料順序
fin_report = fin_report[["Company", "Code", "Industry", "Year", "Season", "announce_day", "EPS", "PE", "Divdend_Rate"]]

In [5]:
#選出財報資料完善者(每期都有)
consider_list = fin_report.Code.unique()[fin_report.groupby('Code', sort = False).apply(lambda x : len(x)) == 17]
fin_report = fin_report[fin_report.Code.isin(consider_list)]

In [6]:
#計算TTM PE
fin_report["TTM_PE"] = fin_report.groupby("Code", group_keys = False, sort = False).apply(lambda x : x.PE.rolling(4).mean())

In [7]:
#定義函數計算EPS同比成長率

def EPS_growth(x):
    x.reset_index(inplace = True)
    temp = np.zeros(len(x))
    for i in range(len(x)):
        if x["Year"][i] == 2013:
            temp[i] = np.nan
        else:
            temp[i] = (x["EPS"][i] - x["EPS"][i - 4]) / x["EPS"][i - 4]
    return(temp)

temp_result = fin_report.groupby("Code", sort = False, as_index = False).apply(EPS_growth) #要把sort設成False合併回原表時順序才不會變

#出來的值shape是(xxx, )，因此要把值抽出來變成1D array，以下用for loop但應該有更好的方法
result = []
for i in range(len(temp_result)):
    result = np.append(result, temp_result[i])
    
fin_report["EPS_growth"] = result

#以下是完全用for loop的作法
# eps_result = []
# for code in fin_report.Code.unique():
#     temp_data = fin_report[fin_report.Code == code]
#     eps_temp_result = EPS_growth(temp_data)
#     eps_result = np.append(eps_result, eps_temp_result)

# fin_report["EPS_growth"] = eps_result

C:\Users\andy5\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\Users\andy5\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [8]:
#計算預期股利率
fin_report["Divdend_Rate"] = fin_report["Divdend_Rate"]/100
fin_report["Expeceted_Dividend_Rate"] = fin_report.groupby("Code", group_keys = False, sort = False).apply(lambda x : x.Divdend_Rate.rolling(4).mean())

In [9]:
fin_report

,Company,Code,Industry,Year,Season,announce_day,EPS,PE,Divdend_Rate,TTM_PE,EPS_growth,Expeceted_Dividend_Rate
0,1101 台泥,1101,水泥工業,2013,1,2013-05-15,0.38,12.60,0.0507,NaN,NaN,NaN
1,1101 台泥,1101,水泥工業,2013,2,2013-08-14,0.70,10.52,0.0515,NaN,NaN,NaN
2,1101 台泥,1101,水泥工業,2013,3,2013-11-14,0.97,9.71,0.0444,NaN,NaN,NaN
3,1101 台泥,1101,水泥工業,2013,4,2014-03-31,0.66,9.38,0.0497,10.5525,NaN,0.049075
4,1101 台泥,1101,水泥工業,2014,1,2014-05-15,0.50,8.79,0.0489,9.6000,0.315789,0.048625
5,1101 台泥,1101,水泥工業,2014,2,2014-08-14,0.87,7.80,0.0509,8.9200,0.242857,0.048475
6,1101 台泥,1101,水泥工業,2014,3,2014-11-14,0.87,7.99,0.0508,8.4900,-0.103093,0.050075
7,1101 台泥,1101,水泥工業,2014,4,2015-03-31,0.70,7.62,0.0574,8.0500,0.060606,0.052000
8,1101 台泥,1101,水泥工業,2015,1,2015-05-15,0.22,8.53,0.0565,7.9850,-0.560000,0.053900
9,1101 台泥,1101,水泥工業,2015,2,2015-08-14,0.61,8.96,0.0639,8.2750,-0.298851,0.057150


In [10]:
fin_report.to_csv("fin_report.csv", index = False)